# Data Cleaning: MEPS Medical Conditions Data (2014-2019)

## Import packages and read data

In [92]:
# Import statements
import pandas as pd
import numpy as np

# To show all lines of output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [93]:
# Reading in the data
data_raw = pd.read_excel('h199.xlsx')

## View the data

In [94]:
# Describing the data
data_raw.describe()
data_raw.columns

# Filter out only columns we need
df = data_raw[['DUPERSID', 'CONDN', 'INJURY']]

,DUID,PID,DUPERSID,CONDN,CONDIDX,PANEL,CONDRN,AGEDIAG,CRND1,CRND2,...,ACCDNWRK,HHNUM,IPNUM,OPNUM,OBNUM,ERNUM,RXNUM,PERWT17F,VARSTR,VARPSU
count,112630.000000,112630.000000,1.126300e+05,112630.000000,1.126300e+05,112630.000000,112630.000000,112630.000000,112630.000000,112630.000000,...,112630.000000,112630.000000,112630.000000,112630.000000,112630.000000,112630.000000,112630.000000,112630.000000,112630.000000,112630.000000
mean,54395.458599,103.138471,5.439556e+07,64.149925,5.439556e+11,21.494442,2.122587,7.899689,-0.266465,-0.216346,...,-0.841108,0.111675,0.031484,0.149383,1.687765,0.064796,1.285918,10850.060774,1565.930827,1.626503
std,40149.731440,13.468809,4.014973e+07,55.829979,4.014973e+11,0.499971,1.223972,20.469450,0.820764,0.864854,...,0.737240,0.945474,0.208545,1.476622,5.801767,0.303860,1.889144,8009.562354,487.947178,0.641028
min,10001.000000,101.000000,1.000110e+07,11.000000,1.000110e+11,21.000000,1.000000,-9.000000,-1.000000,-1.000000,...,-9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1001.000000,1.000000
25%,14667.000000,101.000000,1.466710e+07,21.000000,1.466710e+11,21.000000,1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5602.930770,1086.000000,1.000000
50%,19569.000000,101.000000,1.956910e+07,51.000000,1.956910e+11,21.000000,2.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,9112.861179,1163.000000,2.000000
75%,94866.000000,102.000000,9.486610e+07,91.000000,9.486610e+11,22.000000,3.000000,-1.000000,0.000000,1.000000,...,-1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,13760.992781,2056.000000,2.000000
max,99692.000000,503.000000,9.969210e+07,581.000000,9.969210e+11,22.000000,5.000000,85.000000,1.000000,1.000000,...,3.000000,43.000000,9.000000,140.000000,305.000000,11.000000,40.000000,98538.075709,2117.000000,3.000000


Index(['DUID', 'PID', 'DUPERSID', 'CONDN', 'CONDIDX', 'PANEL', 'CONDRN',
       'AGEDIAG', 'CRND1', 'CRND2', 'CRND3', 'CRND4', 'CRND5', 'INJURY',
       'ACCDNWRK', 'ICD10CDX', 'HHNUM', 'IPNUM', 'OPNUM', 'OBNUM', 'ERNUM',
       'RXNUM', 'PERWT17F', 'VARSTR', 'VARPSU'],
      dtype='object')

In [96]:
# List of column names
list(df.columns)

# Number of unique IDs
df['DUPERSID'].nunique()

# Length of original dataset
len(df)

# Check value counts of injury
df['INJURY'].value_counts(dropna=False)

['DUPERSID', 'CONDN', 'INJURY']

23936

112630

2    105570
1      7060
Name: INJURY, dtype: int64

## Data cleaning

#### Recoding Injury from 1/2 to 1/0

In [126]:
# Change INJURY from 1/2 to 1/0
df.loc[df['INJURY'] == 2] = 0

# Check recoded value counts of injury - good
df['INJURY'].value_counts(dropna=False)

<ipython-input-126-2eff85d87e2f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['INJURY'] == 2] = 0


0    105570
1      7060
Name: INJURY, dtype: int64

#### Aggregating data into a new dataframe

In [168]:
# Aggregate data by DUPERSID
df_agg = df.groupby(by="DUPERSID", as_index=False).agg(
    # Create new vars
    NUM_CONDITIONS = ('CONDN', 'count'),
    INJURY2 = ('INJURY', 'max')) # HERE IS THE PROBLEM

#### Checking data cleaning

In [169]:
# Check new column names
list(df_agg.columns)

# Check length of agg df = # of unique ID's
len(df_agg)
df_agg['DUPERSID'].nunique()

# Check that # unique IDs stayed the same in agg df
df['DUPERSID'].nunique() == df_agg['DUPERSID'].nunique()

# Check that sum of # conditions = number of rows of original df
sum(df_agg["NUM_CONDITIONS"]) == len(df)

# See frequencies of "INJURY"
df_agg['INJURY2'].value_counts()

['DUPERSID', 'NUM_CONDITIONS', 'INJURY2']

5031

5031

True

True

1    5030
0       1
Name: INJURY2, dtype: int64

In [164]:
# Checking against original injury var
# Num of 1's should be 3804

test = df.loc[df['INJURY'] == 1]
test.drop_duplicates(subset ="DUPERSID",
                     keep = False, inplace = True)
len(test)

<ipython-input-164-e9d4951b10da>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop_duplicates(subset ="DUPERSID",


3804

## Next To Do

In [ ]:
# ADD DUPERSID YEAR CODE
# DO FOR ALL YEARS

# CHECK NA's FOR ALL YEAR DATSETS FOR OUR COLUMNS